## Setup

In [13]:
import os

main_path = "/content/drive/MyDrive/Thought Process"
os.chdir(main_path)
os.listdir()

['2024-06-21-code', '2024-06-30-code']

In [14]:
! pip install pyautogen

In [15]:
from datetime import datetime

today = datetime.now().strftime("%Y-%m-%d")
print(today)

2024-06-30


In [16]:
from pathlib import Path
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir = Path(f"{today}-code")
work_dir.mkdir(exist_ok=True)
executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

from autogen import ConversableAgent

# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

In [17]:
! pip install openai

In [18]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [19]:
from openai import OpenAI


class ChatBot:
    def __init__(self, protocol):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.protocol = protocol
        self.history = [{"role": "system", "content": self.protocol}]

    def generate_response(self, prompt: str) -> str:
        self.history.append({"role": "user", "content": prompt})

        completion = self.client.chat.completions.create(
            model="gpt-3.5-turbo", # NOTE: feel free to change it to gpt-4, or gpt-4o
            messages=self.history
        )

        response = completion.choices[0].message.content
        self.history.append({"role": "assistant", "content": response})

        return response

    def get_history(self) -> list:
        return self.history

## Initiate Bot

In [20]:
# Initiate bot
bot = ChatBot(
    protocol="""
    You are a code assistant and executor.

    User will ask you a question about python function.
    You will write the python function with docstring, and type hints.
    You will then execute the python function.
    In the end of the python script, you will execute the functions.
    Last, you will write a reply to the user.
    """
)

# Generate a response for the given input
prompt = input("Ask me a python function: ")

# Loop
while "exit" not in prompt.lower():
    # Checkpoint
    print()
    print()
    print("--------------------------")
    print("Start of a conversation:")
    print()
    print()

    bot.generate_response(prompt)
    message_with_code_block = bot.get_history()[-1]["content"]
    print(message_with_code_block)

    # Generate a reply for the given code.
    reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
    print(reply)

    # Checkpoint
    print()
    print()
    print("--------------------------")
    print("If you want to quit the conversation, type in 'exit'.")
    print()
    print()

    # Continue the conversation
    prompt = input("Ask me a python function: ")


Ask me a python function: i want to know what the 10th fibonacci number is


--------------------------
Start of a conversation:


```python
def fibonacci(n: int) -> int:
    """
    Calculate the nth Fibonacci number.

    Args:
    n: An integer representing the position of the Fibonacci number to calculate.

    Returns:
    The nth Fibonacci number.
    """
    if n <= 0:
        return "Invalid input. Please provide a positive integer greater than 0."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Calculate the 10th Fibonacci number
tenth_fibonacci = fibonacci(10)
```

```python
print(tenth_fibonacci)
```

>>>>>>>> EXECUTING 2 CODE BLOCKS (inferred languages are [python, python])...
exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/content/drive/MyDrive/Thought Process/2024-06-30-code/tmp_code_34630920a7b89b348377d3d